In [1]:
corr_arr = 0.3 # pearson correlation coefficient
vif_arr = 5 # vif coefficient
features_arr = 10 # total number of features to be selected from backward feature selection
iv_upper_limit = 0.5 # upper threshold of iv
iv_lower_limit = 0.02 # lower threshold of iv
seed = 9
class_weight_dict = 'balanced'

In [2]:
# Import Packages 
import sys
import pandas as pd 
import os 
import json
import pickle
import plotly.express as px

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
# os.environ['PYTHONHASHSEED']=str(42)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed)

import os 
cwd = os.getcwd()

# move to previous direc
os.chdir('../')
cwd = os.getcwd()
print(cwd)

# move to data direc
os.chdir('./data')
cwd = os.getcwd()
print(cwd)


from datetime import date
today = date.today()

# Reading Raw Data 
df = pd.read_pickle(f'/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/final_dataset.pkl')
# df2 = pd.read_pickle('')
print(f"raw data file loaded with shape : {df.shape} ")

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Data
raw data file loaded with shape : (646, 243) 


In [3]:
# Import Sklearn Modules 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

# move to previous direc
os.chdir('../')
cwd = os.getcwd()
print(cwd)

# move to code/src direc
os.chdir('./code/src')
cwd = os.getcwd()
print(cwd)

from utils import Utils
from pipeline_blocks import PipelineBlocks
from pipeline import PipelineTypes, PipelineTest

# move to modules direc
os.chdir('./modules')
cwd = os.getcwd()
print(cwd)

os.chdir('/Users/shashankgupta/Documents/code/git_project/plaid_credit/code/src/modules')
cwd = os.getcwd()
print(cwd)

# Import Other Modules
from preprocess import Convert, MissingValues, Outlier, FeatureSelection
from transform import Scaler, Transform, Selection
from modeling import ModelBuild, ModelMetric
from gridsearch import Grid
print(f"custom novo ds modules loaded ..")


# Opening Params Json File

# move to previous direc
os.chdir('../')
cwd = os.getcwd()
print(cwd)

# move to previous direc
os.chdir('../')
cwd = os.getcwd()
print(cwd)

# move to params folder
os.chdir('./params')
cwd = os.getcwd()
print(cwd)

f = open('params.txt')
params_data = json.load(f)
print(f"params file loaded ..")
params_data['model'] = LogisticRegression()

params_data['forward_move'] = True
params_data['params_log_reg']['class_weight'] = class_weight_dict
params_data['pipeline_os']['scale_type']= StandardScaler()
params_data['seed'] = seed
print(f"{params_data}")

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code/src
params file loaded ..
params file loaded ..
{'seed': 42, 'test_size': 0.3, 'target': 'DPD_plus_15', 'corr_thresh': 0.3, 'vif_thresh': 7, 'model': LogisticRegression(), 'forward_move': True, 'num_features': 10, 'bins': 5, 'upper_iv': 0.5, 'lower_iv': 0.02, 'params_log_reg': {'penalty': 'l1', 'random_state': 42, 'solver': 'liblinear', 'class_weight': {0: 0.1, 1: 0.9}}, 'pipeline_os': {'scale_type': StandardScaler(), 'target': 'DPD_plus_15'}}
utils module loaded ..
custom novo ds modules loaded ..
all objects inititated ..

Basic Flow : 

    Preprocess -> Transform -> Model Building -> Metrics 


/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code/src/modules
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code/src/modules
custom novo ds modules loaded ..
all objects inititated ..

Basic Flow : 

    Preprocess -> Transform -> Model

In [4]:
# Object Initiation 
sc_min_max = Scaler(MinMaxScaler())
sc_std_scaler = Scaler(StandardScaler())
sc_robust_scaler = Scaler(RobustScaler())
sc_norm = Scaler(Normalizer())

cv = Convert()
mv = MissingValues()
ot = Outlier()
ut = Utils()

tf = Transform()
sel = Selection()
ft = FeatureSelection()

mb = ModelBuild()
mm = ModelMetric()

pb = PipelineBlocks()
pt = PipelineTypes()
ptest = PipelineTest()

gr = Grid()


print(f"all objects inititated ..")

all objects inititated ..


In [5]:
df.head()

,business_id,lending_business_id,decision_date,drawn_flag,everDPD_15,fico_score,target,loans_flag,payroll_flag,pos_flag,...,sum_credits_grt_1500_6M,ratio_credits_lessthan_100_1M_3M,ratio_credits_lessthan_100_1M_6M,ratio_credits_grt_500_1M_3M,ratio_credits_grt_500_1M_6M,ratio_credits_grt_1500_1M_3M,ratio_credits_grt_1500_1M_6M,txn_each_mth_flag,txn_grt_100_each_mth_flag,txn_flag
0,a2317f88-325b-43ee-9423-6c711856555b,01e38425-5486-4c95-b6b0-de5c57319255,2023-02-02,1,0,721,0,0.0,1.0,1.0,...,71528.95,0.333333,0.20,0.363636,0.200000,0.40,0.235294,1.0,1.0,1.0
1,252c9a3f-338f-43fc-893f-87e4c8184e8c,166d62fb-70ed-4214-8a75-1223dfbbd06e,2023-03-15,1,0,846,0,1.0,0.0,1.0,...,42851.39,0.272727,0.15,0.473684,0.243243,0.25,0.125000,1.0,1.0,1.0
2,420bb9d6-c156-4e29-8060-420fa8753e7f,2c5dbbbc-327b-48ac-a737-73dc0c2a4d26,2023-03-07,1,0,720,0,0.0,0.0,0.0,...,13647.10,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0
3,d9f1ffe7-f652-45ec-bdd6-5d530cf6a24a,588d7169-4d5c-4725-bc69-3e6d5e32d394,2023-02-15,1,0,720,0,0.0,1.0,0.0,...,79422.72,NaN,NaN,0.181818,0.105263,0.10,0.058824,1.0,1.0,1.0
4,588127e8-c54e-42d0-955a-a22e64207a2f,4bbe29b6-f631-4146-829c-880d71809e26,2023-03-24,1,0,687,0,0.0,0.0,0.0,...,1500.00,NaN,NaN,0.333333,0.166667,0.50,0.500000,0.0,0.0,1.0


In [6]:
df = df[df['txn_flag']==1]
df.shape

(637, 243)

In [7]:
df = df.drop(['business_id', 'lending_business_id','decision_date','drawn_flag', 'everDPD_15', 'fico_score','txn_flag'], axis=1)
df.shape

(637, 236)

In [8]:
df['target'].value_counts()

target
0    505
1    132
Name: count, dtype: int64

In [9]:
df_raw = df.copy()
df2 = df.copy()
df_raw.head()

,target,loans_flag,payroll_flag,pos_flag,ecom_flag,third_party_flag,service_flag,shops_flag,ach_flag,total_credit_count_1M,...,count_credits_grt_1500_6M,sum_credits_grt_1500_6M,ratio_credits_lessthan_100_1M_3M,ratio_credits_lessthan_100_1M_6M,ratio_credits_grt_500_1M_3M,ratio_credits_grt_500_1M_6M,ratio_credits_grt_1500_1M_3M,ratio_credits_grt_1500_1M_6M,txn_each_mth_flag,txn_grt_100_each_mth_flag
0,0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,11.0,...,16.0,71528.95,0.333333,0.20,0.363636,0.200000,0.40,0.235294,1.0,1.0
1,0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,12.0,...,15.0,42851.39,0.272727,0.15,0.473684,0.243243,0.25,0.125000,1.0,1.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,1.0,13647.10,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,3.0,...,16.0,79422.72,NaN,NaN,0.181818,0.105263,0.10,0.058824,1.0,1.0
4,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1500.00,NaN,NaN,0.333333,0.166667,0.50,0.500000,0.0,0.0


In [10]:
df_raw = df_raw.fillna(0)
df_raw.head()

,target,loans_flag,payroll_flag,pos_flag,ecom_flag,third_party_flag,service_flag,shops_flag,ach_flag,total_credit_count_1M,...,count_credits_grt_1500_6M,sum_credits_grt_1500_6M,ratio_credits_lessthan_100_1M_3M,ratio_credits_lessthan_100_1M_6M,ratio_credits_grt_500_1M_3M,ratio_credits_grt_500_1M_6M,ratio_credits_grt_1500_1M_3M,ratio_credits_grt_1500_1M_6M,txn_each_mth_flag,txn_grt_100_each_mth_flag
0,0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,11.0,...,16.0,71528.95,0.333333,0.20,0.363636,0.200000,0.40,0.235294,1.0,1.0
1,0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,12.0,...,15.0,42851.39,0.272727,0.15,0.473684,0.243243,0.25,0.125000,1.0,1.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,13647.10,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0
3,0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,3.0,...,16.0,79422.72,0.000000,0.00,0.181818,0.105263,0.10,0.058824,1.0,1.0
4,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1500.00,0.000000,0.00,0.333333,0.166667,0.50,0.500000,0.0,0.0


#### Data Preprocessing

In [11]:
x_train, y_train, x_test, y_test = mb.split_test_train(df_raw, target_column='target', test_size=0.3, random_state = seed)
print(f'{x_train.shape = }', '|' ,f'{y_train.shape = }', '|' ,f'{x_test.shape = }', '|' ,f'{y_test.shape = }')


# copy to df
df = x_train.copy(deep=True)

x_train.shape = (445, 235) | y_train.shape = (445,) | x_test.shape = (192, 235) | y_test.shape = (192,)


In [12]:
# get datatypes frequency
def get_datatypes_freq(df):
    type_dct = {str(k): list(v) for k, v in df.groupby(df.dtypes, axis=1)}
    type_dct_info = {k: len(v) for k, v in type_dct.items()}
    return type_dct, type_dct_info

type_dct, type_dct_info = get_datatypes_freq(df)
type_dct_info

{'float64': 235}

#### Treat missing features

In [13]:
null_df = pd.DataFrame(df.isnull().sum().sort_values(ascending=False),columns = ['null_count'])
null_df = null_df[null_df['null_count']>0]
null_df.shape

(0, 1)

#### Columns except for booleans

In [14]:
# get boolean columns
def findbool(df):
    bool_arr = []
    for col in df.columns: 
        if (len(df[col].unique())<=2):
            bool_arr.append(col)
    return(bool_arr)

bool_col_list = findbool(df)
len(bool_col_list)

type_dct, type_dct_info = get_datatypes_freq(df)
col_list = ( type_dct['float64'])
col_list_excpt_bool = [column for column in col_list if column not in bool_col_list]
len(col_list_excpt_bool)

225

#### Store train data params for substitution in test

In [ ]:
data_params = pd.DataFrame(columns=['feature', 'median', 'lower_limit', 'upper_limit'])
data_params['feature'] = col_list_excpt_bool
data_params['median'] = df[col_list_excpt_bool].median().values
data_params['lower_limit'] = df[col_list_excpt_bool].quantile([0.01, 0.99]).values[0]
data_params['upper_limit'] = df[col_list_excpt_bool].quantile([0.01, 0.99]).values[1]

#### Min Max Capping

In [ ]:
df_info = pd.DataFrame(columns=['feature', 'lower_cap', 'upper_cap', 'lower_values_capped', 'upper_values_capped'])
df_info['feature'] = col_list_excpt_bool
for col in col_list_excpt_bool:
    percentiles = df[col].quantile([0.01, 0.99]).values
    df_info.loc[df_info['feature'] == col, 'lower_cap'] = percentiles[0]
    df_info.loc[df_info['feature'] == col, 'upper_cap'] = percentiles[1]
    df_info.loc[df_info['feature'] == col, 'lower_values_capped'] = df[col][df[col] < percentiles[0]].shape[0]/df.shape[0]
    df_info.loc[df_info['feature'] == col, 'upper_values_capped'] = df[col][df[col] > percentiles[1]].shape[0]/df.shape[0]
    df[col][df[col] < percentiles[0]] = percentiles[0]
    df[col][df[col] > percentiles[1]] = percentiles[1]

In [ ]:
df_info.sort_values(by='lower_values_capped', ascending=False)

#### Standardisation

In [15]:
# reset index
df.reset_index(drop=True, inplace=True)
df_trans = df.copy()
# scaling
transformer = StandardScaler().fit(df[col_list_excpt_bool])

df[col_list_excpt_bool] = pd.DataFrame(transformer.transform(df[col_list_excpt_bool]), columns=col_list_excpt_bool)
df.shape

(445, 235)

In [16]:
len(col_list_excpt_bool)

225

#### IV & WOE

In [ ]:
## iv_woe

## iv_woe

def iv_woe(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})

        
        # Calculate the number of events in each group (bin)
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        
        # Calculate % of events in each group.
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()

        # Calculate the non events in each group.
        d['Non-Events'] = d['N'] - d['Events']
        # Calculate % of non events in each group.
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()

        # Calculate WOE by taking natural log of division of % of non-events and % of events
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF

# remove features on basis of IV
y_train.reset_index(drop=True, inplace=True)
df['target'] = y_train
temp = df.copy()
t1, t2 = iv_woe(temp[np.append(col_list_excpt_bool,['target'])], 'target', bins=5, show_woe=False)
feature_list = list(t1[ (t1['IV']<iv_upper_limit) & (t1['IV']>iv_lower_limit) ]['Variable'].values)
len(feature_list)

#### Correlation & VIF

In [ ]:
# view correlation
corr_df, subset_df = sel.get_correlated_features(df, feature_list, thresh=corr_arr)
corr_df

In [ ]:
# remove correlated features
feature_list = sel.corr_iter(df, np.array(feature_list), thresh=corr_arr)
feature_list = list(feature_list)
len(feature_list)

In [ ]:
# get feature list after iterative VIF elimination
from statsmodels.stats.outliers_influence import variance_inflation_factor

def vif_iter(df, iv, threshold=10):
    vif_data = pd.DataFrame()
    vif_data["feature"] = iv
    vif_data["VIF"] = [variance_inflation_factor(df[iv].values, i) for i in range(len(iv))]
    if len(vif_data[vif_data['VIF'] == np.inf]) > 0:
        feature = vif_data[vif_data['VIF'] == np.inf]['feature'].iloc[0]
        iv.remove(feature)
        vif_iter(df, iv, threshold)
    elif len(vif_data[vif_data['VIF'] > threshold]) > 0:
        feature = vif_data.sort_values(by='VIF', ascending=False)['feature'].iloc[0]
        iv.remove(feature)
        vif_iter(df, iv, threshold)
    vif_data = pd.DataFrame()
    vif_data["feature"] = iv
    vif_data["VIF"] = [variance_inflation_factor(df[iv].values, i) for i in range(len(iv))]
    return iv, vif_data

feature_list, vif_df = vif_iter(df, feature_list, threshold=vif_arr)
len(feature_list)

#### Forward Feature Selection

In [ ]:
logit = LogisticRegression(penalty='l2', solver='liblinear', class_weight = class_weight_dict, random_state = seed)
feat_list = ft.move_feature_selection(df[feature_list], y_train, num_features=features_arr,model=logit,forward = True)
feat_list

In [ ]:
# feat_list = ['card_withdrawal_size_2m',
#  'distinct_debit_txns_2m',
#  'ratio_debit_credit_2m',
#  'ratio_ach_debit_total_debit_2m',
#  'ratio_ach_credit_total_credit_2m',
#  'ratio_pd_freq_total_credit_1m',
#  'avg_running_balance_2nd_m',
#  'distinct_cashins_2m_prev',
#  'ratio_amt_drawn_mom']

#### Fit Model

In [ ]:
# hyperparameters
params_log_reg = {'penalty': 'l2',
                  'random_state': seed,
                  'solver': 'liblinear',
                  'class_weight': class_weight_dict}

# model fit
logreg_model = mb.classification_models(df[feat_list], y_train, params_log_reg, models=['log_reg'])

In [ ]:
# train cv scores
cv_scores = mm.cross_validation(logreg_model, df[feat_list], y_train, scoring='roc_auc', folds=5, seed=seed)
print('CV Scores -',np.round(cv_scores, 2))
print('Mean of CV Scores -',np.round(np.mean(cv_scores),2))

In [ ]:
# train score
mm.model_metrics(logreg_model.predict(df[feat_list]), np.array(y_train), logreg_model.predict_proba(df[feat_list]),tag=1)

#### Feature Importance

In [ ]:
feat_imp = mm.feature_importance(logreg_model, df[feat_list])

In [ ]:
feature_list = list(feat_imp.index)

In [ ]:
feat_imp['abs_imp'] = abs(feat_imp['importance'])
feat_imp.sort_values(by='abs_imp', ascending=False)

#### Test Model

In [ ]:
# reset index
x_test.reset_index(drop=True, inplace=True)

# treat missing values
for col in x_test.columns[x_test.isnull().any(axis=0)]:
    x_test[col].fillna(data_params[data_params['feature'] == col]['median'].values[0], inplace=True)

# min max capping
for col in col_list_excpt_bool:
    lower_cap = data_params[data_params['feature'] == col]['lower_limit'].values[0]
    upper_cap = data_params[data_params['feature'] == col]['upper_limit'].values[0]
    x_test[col][x_test[col] < lower_cap] = lower_cap
    x_test[col][x_test[col] > upper_cap] = upper_cap

# scaling
x_test[col_list_excpt_bool] = pd.DataFrame(transformer.transform(x_test[col_list_excpt_bool]), columns=col_list_excpt_bool)
x_test.shape

In [ ]:
# test cv scores
cv_scores = mm.cross_validation(logreg_model, x_test[feat_list], y_test, scoring='roc_auc', folds=5, seed=seed)
print('CV Scores -',np.round(cv_scores, 2))
print('Mean of CV Scores -',np.round(np.mean(cv_scores),2))

In [ ]:
# test score
mm.model_metrics(logreg_model.predict(x_test[feat_list]), np.array(y_test), logreg_model.predict_proba(x_test[feat_list]),tag = 1)

#### Probability Distribution

In [ ]:
import plotly.express as px

In [ ]:
# copy df
df_all = df_raw.copy()

# reset index
df_all.reset_index(drop=True, inplace=True)

# treat missing values
for col in df_all.columns[df_all.isnull().any(axis=0)]:
    df_all[col].fillna(data_params[data_params['feature'] == col]['median'].values[0], inplace=True)

# min max capping
for col in col_list_excpt_bool:
    lower_cap = data_params[data_params['feature'] == col]['lower_limit'].values[0]
    upper_cap = data_params[data_params['feature'] == col]['upper_limit'].values[0]
    df_all[col][df_all[col] < lower_cap] = lower_cap
    df_all[col][df_all[col] > upper_cap] = upper_cap

# scaling
df_all[col_list_excpt_bool] = pd.DataFrame(transformer.transform(df_all[col_list_excpt_bool]), columns=col_list_excpt_bool)
df_all.shape

predicted_probas = logreg_model.predict_proba(df_all[feat_list])
df_all['proba'] = predicted_probas[:,1:].flatten()
px.histogram(df_all['proba'], nbins=100)

#### Cross Validation

In [ ]:
cv_scores = mm.cross_validation(logreg_model, df_raw[feat_list], df_raw['DPD_plus_15'], scoring='roc_auc', folds=5, seed=seed)
print('CV Scores -',np.round(cv_scores, 2))
print('Mean of CV Scores -',np.round(np.mean(cv_scores),2))

#### Rank Ordering

In [ ]:
# complete dataset
df_all['proba'] = np.round(df_all['proba'], 3)
df_all['DecileRank']= pd.qcut(df_all['proba'], q = 4)
df_stats = pd.DataFrame(df_all.groupby(by='DecileRank')['DPD_plus_15'].mean())
df_stats['volume'] = df_all.groupby(by='DecileRank')['DPD_plus_15'].count()
df_stats

In [ ]:
# train
x_train = df.copy()
x_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)

df_train = x_train.copy()
df_train['DPD_plus_15'] = y_train
df_train['proba'] = logreg_model.predict_proba(x_train[feat_list])[:,1:].flatten()

df_train['proba'] = np.round(df_train['proba'], 3)
df_train['DecileRank']= pd.qcut(df_train['proba'], q = 4)
df_stats = pd.DataFrame(df_train.groupby(by='DecileRank')['DPD_plus_15'].mean())
df_stats['volume'] = df_train.groupby(by='DecileRank')['DPD_plus_15'].count()
df_stats

In [ ]:
# test
x_test.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

df_test = x_test.copy()
df_test['DPD_plus_15'] = y_test
df_test['proba'] = logreg_model.predict_proba(x_test[feat_list])[:,1:].flatten()

df_test['proba'] = np.round(df_test['proba'], 3)
df_test['DecileRank']= pd.qcut(df_test['proba'], q = 4)
df_stats = pd.DataFrame(df_test.groupby(by='DecileRank')['DPD_plus_15'].mean())
df_stats['volume'] = df_test.groupby(by='DecileRank')['DPD_plus_15'].count()
df_stats

#### Saving scale transaformer

In [ ]:
df_trans.head()

In [ ]:
# scaling for selected features
transformer_object = StandardScaler().fit(df_trans[feature_list])

df_trans[feature_list] = pd.DataFrame(transformer_object.transform(df[feature_list]), columns=feature_list)
df_trans.shape

In [ ]:
# # save transaformer
# file_name = 'scale_train_object.pkl'
# path = '/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Models/'
# pickle.dump(transformer_object, open(path + file_name, 'wb'))

In [ ]:
df_test['DPD_plus_15'].sum()/len(df_test)

In [ ]:
df_test.head()

In [ ]:
def target_bucket(x):
    if x > 0.56:
        return 4
    if x > 0.431:
        return 3
    if x > 0.255:
        return 2
    return 1

In [ ]:
def prob_bucket(x):
    if x > 0.8:
        return '>0.8'
    if x > 0.6:
        return '0.6-0.8'
    if x > 0.5:
        return '0.5-0.6'
    return '<0.5'


In [ ]:
df_all['prob_bucket'] = df_all['proba'].apply(lambda x: prob_bucket(x))


In [ ]:
df_all['proba_bucket'] = df_all['proba'].apply(lambda x: target_bucket(x))

In [ ]:
df_all.groupby(['proba_bucket','emi_no'],as_index=False)['DPD_plus_15'].agg(['sum','count']).sort_values(['proba_bucket','emi_no'])

In [ ]:
df_all.groupby(['proba_bucket','DPD_plus_3'],as_index=False)['DPD_plus_15'].agg(['sum','count']).sort_values(['proba_bucket','DPD_plus_3'])

In [ ]:
df_all.groupby('emi_no')['DPD_plus_15'].agg(['sum','count']).sort_values('emi_no')

In [ ]:
df_all.groupby(['pilot_flag','pilot_period_flag','emi_no'])['DPD_plus_15'].agg(['sum','count']).sort_values(['pilot_flag','pilot_period_flag','emi_no'])

#### prob bucket

In [ ]:
df_all.groupby(['prob_bucket'],as_index=False)['DPD_plus_15'].agg(['sum','count']).sort_values(['prob_bucket'])

In [ ]:
print(df_test.shape)
print(df_test[df_test['proba']>0.6].shape)
print(df_test[df_test['proba']>0.7].shape)
print(df_test[df_test['proba']>0.8].shape)
print(df_test[df_test['proba']>0.9].shape)

In [ ]:
df_test.groupby(['emi_no'],as_index=False)['DPD_plus_15'].agg(['sum','count'])

In [ ]:
print('Delinquency rate for (Prob > {}) - {} & proportion = {}%'.format(0.6,np.round(df_test.loc[df_test['proba']>0.6,'DPD_plus_15'].sum()/len(df_test[df_test['proba']>0.6]),2), np.round(len(df_test[df_test['proba']>0.6])*100/len(df_test),2)))
print('Delinquency rate for (Prob > {}) - {} & proportion = {}%'.format(0.7,np.round(df_test.loc[df_test['proba']>0.7,'DPD_plus_15'].sum()/len(df_test[df_test['proba']>0.7]),2), np.round(len(df_test[df_test['proba']>0.7])*100/len(df_test),2)))
print('Delinquency rate for (Prob > {}) - {} & proportion = {}%'.format(0.8,np.round(df_test.loc[df_test['proba']>0.8,'DPD_plus_15'].sum()/len(df_test[df_test['proba']>0.8]),2), np.round(len(df_test[df_test['proba']>0.8])*100/len(df_test),2)))
print('Delinquency rate for (Prob > {}) - {} & proportion = {}%'.format(0.9,np.round(df_test.loc[df_test['proba']>0.9,'DPD_plus_15'].sum()/len(df_test[df_test['proba']>0.9]),2), np.round(len(df_test[df_test['proba']>0.9])*100/len(df_test),2)))

In [ ]:
df_all['amount_debited_2m'].mean()

#### Save model object

In [ ]:
# # save data parameters
# data_params = data_params[data_params['feature'].isin(feat_list)]
# file_name = 'data_params_v2.pkl'
# path = '/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Models/'
# data_params.to_pickle(path + file_name)


# # save logit model
# file_name = 'redec_3.pkl'
# path = '/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Models/'
# pickle.dump(logreg_model, open(path + file_name,'wb'))

#### Scoring

In [ ]:
df_all.head()

In [ ]:
df_all.groupby('rs2_bin')['DPD_plus_15'].agg(['sum','count'])

In [ ]:
df_all.groupby('rs2_bin')['proba'].median()

In [ ]:
def buckets(x):
    if x > 0.58:
        return 1
    if x > 0.46:
        return 2
    if x > 0.35:
        return 3
    if x > 0.20:
        return 4
    return 5

In [ ]:
# def buckets(x):
#     if x > 0.587:
#         return 1
#     if x > 0.488:
#         return 2
#     if x > 0.383:
#         return 3
#     if x > 0.266:
#         return 4
#     return 5

In [ ]:
df_all['redec_bucket'] = df_all['proba'].apply(lambda x: buckets(x))

In [ ]:
df_all.groupby('redec_bucket')['DPD_plus_15'].agg(['sum','count']).to_clipboard()

In [ ]:
df_all['redec_score'] = (1 - df_all['proba'])*1000

In [ ]:
df_all.head()